# HW 3 - Trailing Stops

Note - This code extends the HW2 code submitted previously. 



1.  I have used the similar logic of HW2 and added new field `returnVal` in the aggregate tables for all currency pairs.

2. In the output db, we can see 3 types of tables for all currency pairs.   

    *   `<currency_pair>_raw` - This has raw data of all currency pairs
    *   `<currency_pair>_agg` - This has aggregated data of all currency pairs           
    Fields -

    inserttime, period, max, min, mean, vol, fd, returnVal

             1.  inserttime
             2.  period 
             3.  max 
             4.  min
             5.  mean
             6.  vol
             7.  fd
             8.  returnVal 

    *   `<currency_pair>_output` - This has output data of all currency pairs

          Fields -
             1.  window - Every 60 minutes interval window (T10 is window = 1, T20 is window = 2....) 
             2.  balance - Currency units after end of 60 minute period
             3.  returnVal - Return on the currency invested in the current window period
             4.  position - SHORT or LONG 



3. I have added 4 new functions since Hw2, `calculateReturn()`, `getReturnOfLast10()`, `initialize_output_tables()`, `fill_output_data_tables()`

4. All the functions are documented and code is commented so that reviewer will get clear idea of what I am doing in the assignment.

5. I have also converted all the database tables into .csv files in order to submit as mentioned in the assignment document.

6. The critical part of logic is written in  are in the fill_output_data_tables() function.  
   
7. All the changes of this assignment are commented with HW3 Code Changes.

In [1]:
# Install polygon-api-client library
!pip install polygon-api-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import required libraries
import datetime
import time
from polygon import RESTClient
from sqlalchemy import create_engine 
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor

# Importing APIKEY from polygon_api library
from polygon_api import PolygonAPIKEY

In [3]:
'''
THIS IS MAX CODE
'''
# We can buy, sell, or do nothing each time we make a decision.
# This class defies a nobject for keeping track of our current investments/profits for each currency pair
class portfolio(object):
    def __init__(self,from_,to):
        # Initialize the 'From' currency amont to 1
        self.amount = 1
        self.curr2 = 0
        self.from_ = from_
        self.to = to
        # We want to keep track of state, to see what our next trade should be
        self.Prev_Action_was_Buy = False
    
    # This defines a function to buy the 'To' currency. It will always buy the max amount, in whole number
    # increments
    def buy_curr(self, price):
        if self.amount >= 1:
            num_to_buy = floor(self.amount)
            self.amount -= num_to_buy
            self.Prev_Action_was_Buy = True
            self.curr2 += num_to_buy*price
            print("Bought %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_buy,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the original currency (%s) to make another buy." % self.from_)
    # This defines a function to sell the 'To' currency. It will always sell the max amount, in a whole number
    # increments
    def sell_curr(self, price):
        if self.curr2 >= 1:
            num_to_sell = floor(self.curr2)
            self.amount += num_to_sell * (1/price)
            self.Prev_Action_was_Buy = False
            self.curr2 -= num_to_sell
            print("Sold %d worth of the target currency (%s). Our current profits and losses in the original currency (%s) are: %f." % (num_to_sell,self.to,self.from_,(self.amount-1)))
        else:
            print("There was not enough of the target currency (%s) to make another sell." % self.to)   

In [4]:
#---------------------------------- HW2 CODE CHANGES --------------------------------------------
'''
Function to calculate 100 Keltner Upper Bands
'''
def calculateKeltnerChannelUB(mean, VOL):
  UB100values = []
  for n in range(1,101):
    UB100values.append((mean+ n*0.025*VOL))
  return UB100values

#function to calculate 100 Keltner Lower Bands
def calculateKeltnerChannelLB(mean, VOL):
  LB100values = []
  for n in range(1,101):
    LB100values.append((mean - n*0.025*VOL))
  return LB100values

'''
Function that counts N i.e the number of times a price crosses a Keltner Channel from lastPrice to currentPrice

Arguments we pass are -
# lastPrice: previous currency price
# currentPrice: current currency price 
# UB (Upper Band): 100 Keltner Upper Bands
# LB (Lower Band): 100 Keltner Lower Bands
'''
def countBandsCrossing(lastPrice, currentPrice, UB, LB):
  '''
  Step 1: We find the startPrice and endPrice so as to cound all the bands lying in between these two prices
          start price is the price which is min of lastPrice and currentPrice and end price is the larger price
  Step 2: We calculate totalBands i.e 200 bands
  Step 3: We calculate filteredTotalBands which are the bands that the price changes crosses
  Step 4: We return the length of filteredTotalBands which is the count of bands crossed from the lastPrice to currentPrice
  
  '''
  # Finding startPrice and endPrice to find all the bands lying in between the given price change
  startPrice = min(lastPrice, currentPrice)
  endPrice = max(lastPrice, currentPrice)

  # totalBands are the 200 (LB + UB) in sorted order
  totalBands = LB[::-1] + UB

  # filteredTotalBands are the bands lying between the startPrice and endPrice
  filteredTotalBands = filter(lambda x: endPrice>x> startPrice, totalBands)

  # In order to return the count of bands crossed we simply find the len(filteredTotalBands)
  return len(list(filteredTotalBands))

#function to calculate FD (Fractal Dimension) using formula FD = N/VOL
#function will return 0 if the VOL is 0 (In order to avoid divide by 0 error) 
def calculateFD(N, VOL):
  if VOL == 0:
    return 0
  else:
    return N/VOL

#---------------------------------- HW2 CODE CHANGES --------------------------------------------


In [ ]:
#---------------------------------- HW3 CODE CHANGES --------------------------------------------

'''
Calculates the return according to the formula 𝑟𝑖 = (𝑃𝑖 − 𝑃𝑖−1)⁄(𝑃𝑖−1)

Arguments:
  - currentPrice: Current mean price of currency pair
  - lastPrice: Last mean price of currency pair

Returns:
  - return 𝑟𝑖
'''
def calculateReturn(currentPrice, lastPrice):
  return (currentPrice-lastPrice)/lastPrice


'''
Returns sum of last 10 intervals returnVal

Arguments:
  - rows: sql rows object which has last 10 rows from corresponding aggregated table

Returns:
  - sum of 𝑟𝑖
'''
def getReturnOfLast10(rows):
  returnVal = 0
  for row in rows:
    if row.returnVal:
      returnVal += row.returnVal
  return returnVal

#---------------------------------- HW3 CODE CHANGES --------------------------------------------

In [5]:
from ast import Pass
# Function slightly modified from polygon sample code to format the date string 
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

# Function which clears the raw data tables once we have aggregated the data in a 6 minute interval
def reset_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("DROP TABLE "+curr[0]+curr[1]+"_raw;"))
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

# This creates a table for storing the raw, unaggregated price data for each currency pair in the SQLite database
def initialize_raw_data_tables(engine,currency_pairs):
    with engine.begin() as conn:
        for curr in currency_pairs:
            conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_raw(ticktime text, fxrate  numeric, inserttime text);"))

            
# This creates a table for storing the (6 min interval) aggregated price data for each currency pair in the SQLite database            
def initialize_aggregated_tables(engine,currency_pairs):
  with engine.begin() as conn:
    for curr in currency_pairs:
      # Initializes aggregate table for all currency pairs with fields inserttime, period, max, min, mean, vol and fd
      conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_agg (inserttime text, period numeric, max numeric, min numeric, mean numeric, vol numeric, fd numeric, returnVal numeric);"))


def aggregate_raw_data_tables(engine, currency_pairs, period, N, currencyCheck):
  # Initialize UB and LB dictionary which has currency pairs as key and stores bands based on mean and vol
  UB = {}
  LB = {}
  with engine.begin() as conn:
    for curr in currency_pairs:
      if currencyCheck[curr[0]+curr[1]] == True:
        with engine.begin() as conn:
          # Calculates mean, max, and min from raw tables which has price data of last 6 mins period
          result = conn.execute(text("SELECT AVG(fxrate) as mean, MIN(fxrate) as min, MAX(fxrate) as max FROM "+curr[0]+curr[1]+"_raw;"))
          for row in result:
            mean = row.mean
            min = row.min
            max = row.max
            vol = max-min

          # Calculate UB and LB for the currency pair and storing it into dict
          UB[curr[0]+curr[1]] = calculateKeltnerChannelUB(mean, vol)
          LB[curr[0]+curr[1]] = calculateKeltnerChannelLB(mean, vol)

          date_res = conn.execute(text("SELECT MAX(ticktime) as last_date FROM " +curr[0]+curr[1]+"_raw;"))
          for row in date_res:
            last_date = row.last_date

          # If it is the 1st period, we skip adding FD value in table 
          if period == 1:
            conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_agg (inserttime, period, max, min, mean, vol) VALUES (:inserttime, :period, :max, :min, :mean, :vol);"),[{"inserttime":last_date, "period": period, "max": max, "min": min, "mean": mean, "vol": vol}])
          else:
            fd = calculateFD(N[curr[0]+curr[1]],vol)
            result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_agg ORDER BY rowid DESC LIMIT 1;"))
            for row in result:
              lastMean = row.mean
            returnVal = calculateReturn(mean, lastMean)
            conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_agg (inserttime, period, max, min, mean, vol, fd, returnVal) VALUES (:inserttime, :period, :max, :min, :mean, :vol, :fd, :returnVal );"),[{"inserttime":last_date, "period": period, "max": max, "min": min, "mean": mean, "vol": vol, "fd": fd, "returnVal":returnVal}])

  # Returning Keltner Channel Upper bound and Lower bound to the main function
  return UB, LB

In [ ]:
#---------------------------------- HW3 CODE CHANGES --------------------------------------------
# This creates an output table for storing the data after 60 min interval for each currency pair in the SQLite database     
# The table has window, balance, returnVal and position fields 
def initialize_output_tables(engine,currency_pairs):
  with engine.begin() as conn:
    for curr in currency_pairs:
      # Initializes aggregate table for all currency pairs with fields inserttime, period, max, min, mean, vol and fd
      conn.execute(text("CREATE TABLE "+curr[0]+curr[1]+"_output (window numeric, balance numeric, returnVal numeric, position text);"))

#---------------------------------- HW3 CODE CHANGES --------------------------------------------

In [ ]:
#---------------------------------- HW3 CODE CHANGES --------------------------------------------

def fill_output_data_tables(engine, currency_pairs, window, currencyCheck):
  '''
    Function called every 60 minutes to make a decision on current open positions and fill the output data tables
    
    Arguments:
      - engine: Engine object from sqlalchemy
      - currency_pairs: Nested list of each currency pair
      - window: current window count
      - currencyCheck: It is a dictionary which has keys as currencyPair and values as True or False
    Returns:
      - currencyCheck: Updated currencyCheck dict
  '''

  #longCurrency List (BUY)
  longCurrency = ["EURUSD", "GBPEUR", "USDJPY", "USDCZK", "USDINR"]
  #shortCurrency List (SELL)
  shortCurrency = [ "AUDUSD", "USDCAD", "USDCNY", "USDPLN", "USDMXN"]


  for curr in currency_pairs:
    # We will check if currency pair is set to True in the currencyCheck dict which will let us know weather to close the position or continue
    if currencyCheck[curr[0]+curr[1]] == True:

      # setting position based on longCurrency list
      if curr[0]+curr[1] in longCurrency:
        position = 'LONG'
      else:
        position = 'SHORT'

      with engine.begin() as conn:
        # Fetching last 10 rows from aggregate table to get the return value
        result = conn.execute(text("SELECT *  FROM "+curr[0]+curr[1]+"_agg ORDER BY rowid DESC LIMIT 10;"))
        # R10 stores the sum of last 10 returnVal 
        R10 = getReturnOfLast10(result)

        # At T10, cutoff value to use is 0.250%
        if window == 1:
          balance = 100
          if position == 'LONG':
            # Long condition -> , a profitable trade has a positive return but we have a tolarence of 0.250%
            # 0.250% = 0.0025
            if R10 >= -0.0025:
              balance = balance + 100 + R10
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition -> a profitable trade has a negative return. 
            if R10 <= 0.0025:
              balance = balance + 100 + R10
            # If not profitable, we will close the position and set the currencyCheck flag to false
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False
        
        # At T20, cutoff value to use is 0.150%
        elif window == 2:
          # The below line fetches the last balance of the currency pair of last 60 min period
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0015:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False
          else:
            # Short condition
            if R10 <= 0.0015:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False


        # At T30, value to use is 0.100%
        elif window == 3:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.001:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.001:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False


        # At T40, value to use is 0.050%
        elif window == 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False


        # After T40, value to use is 0.050%
        elif window > 4:
          result = conn.execute(text("SELECT * FROM " +curr[0]+curr[1]+"_output WHERE window = " + str(window-1)+";"))
          for row in result:
            balance = row.balance

          if position == 'LONG':
            if R10 >= -0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False
          else:
            if R10 <= 0.0005:
              balance = balance + 100 + R10
            else:
              balance = balance + R10
              currencyCheck[curr[0]+curr[1]] = False
          
        conn.execute(text("INSERT INTO " +curr[0]+curr[1]+"_output (window, balance, returnVal, position) VALUES (:window, :balance, :returnVal, :position);"),[{"window":window, "balance": balance, "returnVal": R10, "position": position}])

  return currencyCheck

#---------------------------------- HW3 CODE CHANGES --------------------------------------------

In [ ]:
# This main function repeatedly calls the polygon api every 1 seconds for 24 hours 
# and stores the results.
def main(currency_pairs):
    # The api key given by the professor is fetched from the library imported in the 1st cell 
    polygonAPIObject = PolygonAPIKEY()
    key = polygonAPIObject.fetchKEY()
    
    # Number of list iterations - each one should last about 1 second
    count = 0
    agg_count = 0
    # Counter to keep track of 10 intervals which is after every 60 minutes
    decision_count = 0

    # currencyCheck dictionary has keys as currencyPair and values as True or False
    # By default all are set to True but if we decide not to invest more and stop the position, we set this to False
    currencyCheck = {}
    
    # Create an engine to connect to the database; setting echo to false should stop it from logging in std.out
    engine = create_engine("sqlite:///final.db", echo=False, future=True)
    
    # Create the needed tables in the database
    initialize_raw_data_tables(engine,currency_pairs)
    initialize_aggregated_tables(engine,currency_pairs)
    # Create the output tables for all currency pairs
    initialize_output_tables(engine,currency_pairs)
    
    # Open a RESTClient for making the api calls
    client = RESTClient(key)
   
    # counter for period 
    period_count = 0 
    # counter for window 
    window_count = 0
    # dictionary to store N values for all currency pairs
    N = {}
    UB = {}
    LB = {}

    # stores last price of currencypairs
    lastPrice = {}

    # Loop that runs until the total duration of the program hits 10 hours. 
    # 36000 seconds = 10 hours
    while count < 36000:
      
      # Make a check to see if 6 minutes has been reached or not
      if agg_count == 360:
        # Aggregate the data and clear the raw data tables
        period_count+=1
        UB, LB = aggregate_raw_data_tables(engine, currency_pairs, period_count, N, currencyCheck)
        reset_raw_data_tables(engine,currency_pairs)
        agg_count = 0
        N = {}
        lastPrice = {}
        
      # Only call the api every 1 second, so wait here for 0.75 seconds, because the 
      # code takes about .15 seconds to run
      time.sleep(0.75)

      # Only called when 10 periods of 6 mins are done i.e after every 60 mins
      if decision_count == 3600:
        window_count +=1
        currencyCheck = fill_output_data_tables(engine, currency_pairs, window_count, currencyCheck)
        decision_count = 0
      
      # Increment the counters
      count += 1
      agg_count +=1
      decision_count +=1

      # Loop through each currency pair
      for currency in currency_pairs:
        # Set the input variables to the API
        from_ = currency[0]
        to = currency[1]

        # initializing currencyCheck for all currency pair with True
        if from_+to not in currencyCheck:
          currencyCheck[from_+to] = True

        # Run the below logic only when currencyCheck of the corresponding currency pair is set to True 
        if currencyCheck[from_+to] == True:
          # Call the API with the required parameters
          try:
              resp = client.forex_currencies_real_time_currency_conversion(from_, to, amount=100, precision=2)
          except:
              continue

          # This gets the Last Trade object defined in the API Resource
          last_trade = resp.last

          # Format the timestamp from the result
          dt = ts_to_datetime(last_trade["timestamp"])

          # Get the current time and format it
          insert_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
          
          # Calculate the price by taking the average of the bid and ask prices
          avg_price = (last_trade['bid'] + last_trade['ask'])/2

          # In the 1st period, UB and LB will be an empty dictionary so this condition will be false for 1st period
          if UB!={} and LB!={}:

            # From 2nd period to 100th period
            # if currency[0]+currency[1] key is in lastPrice, it means we have a last price to send in the countBandsCrossing() function and we update lastPrice with the current price which is the avg_price
            if currency[0]+currency[1] in lastPrice:
              N[currency[0]+currency[1]] += countBandsCrossing(lastPrice[currency[0]+currency[1]], avg_price, UB[currency[0]+currency[1]], LB[currency[0]+currency[1]])    
              lastPrice[currency[0]+currency[1]] = avg_price
            
            # if currency[0]+currency[1] key not in lastPrice and N, we initialize here
            else:
              lastPrice[currency[0]+currency[1]] = avg_price
              N[currency[0]+currency[1]] = 0

          # Write the data to the SQLite database, raw data tables
          with engine.begin() as conn:
              conn.execute(text("INSERT INTO "+from_+to+"_raw(ticktime, fxrate, inserttime) VALUES (:ticktime, :fxrate, :inserttime)"),[{"ticktime": dt, "fxrate": avg_price, "inserttime": insert_time}])

In [ ]:
# A dictionary defining the set of currency pairs we will be pulling data for
currency_pairs = [["AUD","USD",[],portfolio("AUD","USD")],
                  ["GBP","EUR",[],portfolio("GBP","EUR")],
                  ["USD","CAD",[],portfolio("USD","CAD")],
                  ["USD","JPY",[],portfolio("USD","JPY")],
                  ["USD","MXN",[],portfolio("USD","MXN")],
                  ["EUR","USD",[],portfolio("EUR","USD")],
                  ["USD","CNY",[],portfolio("USD","CNY")],
                  ["USD","CZK",[],portfolio("USD","CZK")],
                  ["USD","PLN",[],portfolio("USD","PLN")],
                  ["USD","INR",[],portfolio("USD","INR")]]

# Run the main data collection loop
main(currency_pairs)

In [ ]:
# Code to convert db tables into csv files

import pandas as pd
import sqlite3
conn = sqlite3.connect('final.db', isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)

for curr in currency_pairs:
  sql = "SELECT * FROM "+curr[0]+curr[1]+"_agg"
  db_df = pd.read_sql_query(sql, conn)
  path = curr[0]+curr[1]+'_agg.csv'
  db_df.to_csv(path, index=False)

for curr in currency_pairs:
  sql = "SELECT * FROM "+curr[0]+curr[1]+"_output"
  db_df = pd.read_sql_query(sql, conn)
  path = curr[0]+curr[1]+'_output.csv'
  db_df.to_csv(path, index=False)